# Hugging Face GPT2 fine-tuning

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.7 MB/s 
     |████████████████████████████████| 182 kB 84.3 MB/s 
     |████████████████████████████████| 7.6 MB 75.5 MB/s 


In [3]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 4.6 MB/s 
     |████████████████████████████████| 115 kB 90.3 MB/s 
     |████████████████████████████████| 212 kB 84.0 MB/s 
     |████████████████████████████████| 127 kB 78.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


## Hugging Face fine-tuning

From https://huggingface.co/docs/datasets/nlp_load:

In [ ]:
from datasets import load_dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATASET_DIRECTORY = "/content/drive/MyDrive/Movie Script Generator/dataset/txt_pruned"

In [ ]:
dataset = load_dataset("text", data_dir=DATASET_DIRECTORY)

Resolving data files:   0%|          | 0/3003 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-a787a67b7bc4768e/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 18826310
    })
})

In [ ]:
dataset["train"][10]

{'text': 'ACTION | \ton a tray table.  Crumpled Czech bills and coins are on it. '}

In [ ]:
dataset["text"]

KeyError: ignored

From https://huggingface.co/docs/transformers/training:

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
tokenizer.add_special_tokens({'pad_token': '<|pad|>'}) # added so that I don't get an error when tokenizing

1

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [ ]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

  0%|          | 0/18827 [00:00<?, ?ba/s]

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 18826310
    })
})

In [ ]:
tokenized_dataset["train"]

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 18826310
})

In [ ]:
tokenized_dataset_train = tokenized_dataset["train"]

### Training

In [ ]:
from transformers import GPT2Model

In [ ]:
model = GPT2Model.from_pretrained("gpt2")

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [ ]:
from transformers import TrainingArguments

In [ ]:
training_args = TrainingArguments(output_dir="model_checkpoints")

### TODO: See how to evaluate the model

In [ ]:
from transformers import Trainer

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `GPT2Model.forward` and have been ignored: text. If text are not expected by `GPT2Model.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 18826310
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 7059867


RuntimeError: ignored

This is the 3rd or the 4th error in a row, so now I'm going to go with PyTorch fine-tuning approach. (https://stackoverflow.com/questions/72604790/how-to-train-gpt2-with-huggingface-trainer)

## PyTorch fine-tuning

In [ ]:
import os

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "" # attempt to not use the GPU

From https://huggingface.co/docs/datasets/nlp_load:

In [ ]:
from datasets import load_dataset

In [ ]:
DATASET_DIRECTORY = "dataset/txt_pruned"

In [ ]:
dataset = load_dataset("text", data_dir=DATASET_DIRECTORY)

In [ ]:
dataset

In [ ]:
dataset["train"][10]

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [ ]:
tokenizer.add_special_tokens({'pad_token': '<|pad|>'}) # added so that I don't get an error when tokenizing

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [ ]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

In [ ]:
tokenized_dataset

In [ ]:
tokenized_dataset["train"]

In [ ]:
tokenized_dataset_train = tokenized_dataset["train"]

Let's post-process the dataset:

In [ ]:
tokenized_dataset_train = tokenized_dataset_train.remove_columns(["text"])

In [ ]:
tokenized_dataset_train

In [ ]:
tokenized_dataset_train.set_format("torch")

Let's create a DataLoader:

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_dataloader = DataLoader(tokenized_dataset_train, shuffle=True, batch_size=1)

In [ ]:
from transformers import GPT2Model

In [ ]:
model = GPT2Model.from_pretrained("gpt2")

In [ ]:
model.resize_token_embeddings(len(tokenizer))

In [ ]:
from torch.optim import AdamW

In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
from transformers import get_scheduler

In [ ]:
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
import torch

In [ ]:
device = "cpu" # hardcoding cpu since I want it to run on the CPU, so I don't get out of memory errors
model.to(device)

Now let's write the training loop:

In [ ]:
from tqdm.auto import tqdm

In [ ]:
progress_bar = tqdm(range(num_training_steps))

model.train()

for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

I'm giving up on this approach as well, as it has given me many errors. (Note: I tested it locally, so there's no output here)

## PyTorch fine-tuning #2

This Jupyter notebook is based on [this Jupyter notebook](https://colab.research.google.com/drive/13dZVYEOMhXhkXWfvSMVM1TTtUDrT6Aeh?usp=sharing#scrollTo=U_XJVIetKN-h).

Let's first define the dataset:

In [4]:
import os

import torch
from torch.utils.data import Dataset

from transformers import GPT2Tokenizer

In [5]:
class GPT2Dataset(Dataset):

    def __init__(self, dataset_dir, max_length=768):
        # stores each line of the movie script file as a separate sequence
        self.tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')
        self.dataset_dir = dataset_dir
        self.max_length = max_length

        genre_subfolders = [subfolder_name for subfolder_name in os.listdir(dataset_dir)]
        self.genre_subfolders = genre_subfolders
        
        # indicies used to load movie lines on demand (otherwise 25 GB of RAM is not enough)
        self.genre_subfolder_index = 0
        self.movie_script_in_genre_subfolder_index = 0
        self.text_line_in_movie_script_index = 0

        # stores the total number of movie lines across all movie scripts
        self.total_number_of_movie_lines = 0
    
        current_cumulative_number_of_movie_lines = 0
        # a dictionary that stores "global" movie line indices ranges (across all genres and all movie scripts)
        self.genre_movie_script_pairs_global_end_index = {}
        
        for genre_subfolder in self.genre_subfolders:
            genre_subfolder_path = self.dataset_dir + "/" + genre_subfolder
            txt_files_names = [txt_file_name for txt_file_name in os.listdir(genre_subfolder_path)]
            
            for txt_file_name in txt_files_names:
                path_to_movie_script = genre_subfolder_path + "/" + txt_file_name
                with open(path_to_movie_script) as movie_script_file:
                    movie_script_all_lines = movie_script_file.readlines()
                    
                    genre_movie_script_pair = str(genre_subfolder) + "/" + str(txt_file_name)
                    self.genre_movie_script_pairs_global_end_index[current_cumulative_number_of_movie_lines + len(movie_script_all_lines)] = genre_movie_script_pair
                    
                    self.total_number_of_movie_lines += len(movie_script_all_lines)
                    current_cumulative_number_of_movie_lines = current_cumulative_number_of_movie_lines + len(movie_script_all_lines)
        
        # prints for debugging purposes
        #for global_end_index, genre_movie_script_pair in self.genre_movie_script_pairs_global_end_index.items():
            #print(str(global_end_index) + ": " + str(genre_movie_script_pair))
            
    def __len__(self):
        return self.total_number_of_movie_lines
    
    def __getitem__(self, idx):
        target_genre_movie_script_pair = None
        previous_global_end_index = None
        trail_global_end_index = 0 # stores the global end index of the previous genre movie script pair
        
        for global_end_index, genre_movie_script_pair in self.genre_movie_script_pairs_global_end_index.items():
            if (idx < global_end_index):
                target_genre_movie_script_pair = genre_movie_script_pair
                previous_global_end_index = trail_global_end_index
                break
            trail_global_end_index = global_end_index
        
        path_to_target_movie_script = self.dataset_dir + "/" + target_genre_movie_script_pair
        # prints for debugging purposes
        #print("path_to_target_movie_script:")
        #print(path_to_target_movie_script)
        target_movie_script_line_index = idx - previous_global_end_index - 1
        target_movie_script_line = None
        with open(path_to_target_movie_script, "r") as movie_script_file:
            movie_script_file_all_lines = movie_script_file.readlines()
            target_movie_script_line = movie_script_file_all_lines[target_movie_script_line_index]
        
        # prints for debugging purposes
        #print("target_movie_script_line_index:")
        #print(target_movie_script_line_index)
        #print("target_movie_script_line:")
        #print(target_movie_script_line)
        encoded_movie_script_line = self.tokenizer(target_movie_script_line, truncation=True, max_length=self.max_length, padding="max_length")
        return torch.tensor(encoded_movie_script_line['input_ids']), torch.tensor(encoded_movie_script_line['attention_mask'])

Let me mount my Google Drive:

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
DATASET_DIRECTORY = "/content/drive/MyDrive/Movie Script Generator/dataset/txt_preprocessed_v2"

Now let's write the data loader code and split the dataset into **train** and **val** subsets:

In [8]:
from torch.utils.data import random_split

In [9]:
dataset = GPT2Dataset(DATASET_DIRECTORY, max_length=768)

# Split into training and validation sets
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


93,004 training samples
10,334 validation samples


In [10]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [11]:
batch_size = 8

In [12]:
# Create the DataLoaders for our training and validation datasets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

## Fine-tuning code

In [13]:
from transformers import GPT2Config, GPT2LMHeadModel

In [14]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
# I'm not really doing anything with the config buheret
configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)

# instantiate the model
model = GPT2LMHeadModel.from_pretrained("gpt2", config=configuration)

# this step is necessary because I've added some tokens (bos_token, etc) to the embeddings
# otherwise the tokenizer and model tensors won't match up
model.resize_token_embeddings(len(tokenizer))

# Tell pytorch to run this model on the GPU.
device = torch.device("cuda")
model.cuda()

# Set the seed value all over the place to make this reproducible.
#seed_val = 42

#random.seed(seed_val)
#np.random.seed(seed_val)
#torch.manual_seed(seed_val)
#torch.cuda.manual_seed_all(seed_val)

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50259, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [16]:
# some parameters I cooked up that work reasonably well

epochs = 5
learning_rate = 5e-4
warmup_steps = 1e2
epsilon = 1e-8

# this produces sample output every 100 steps
sample_every = 100

In [17]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [18]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
optimizer = AdamW(model.parameters(),
                  lr = learning_rate,
                  eps = epsilon
                )

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [19]:
# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
# This changes the learning rate as the training loop progresses
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = warmup_steps, 
                                            num_training_steps = total_steps)

In [20]:
import time
import datetime
import random

In [21]:
def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

In [ ]:
total_t0 = time.time()

training_stats = []

model = model.to(device)

for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        model.zero_grad()        

        outputs = model(  b_input_ids,
                          labels=b_labels, 
                          attention_mask = b_masks,
                          token_type_ids=None
                        )

        loss = outputs[0]  

        batch_loss = loss.item()
        total_train_loss += batch_loss

        # Get sample every x batches.
        if step % sample_every == 0 and not step == 0:

            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(step, len(train_dataloader), batch_loss, elapsed))

            model.eval()

            sample_outputs = model.generate(
                                    #bos_token_id=random.randint(1,30000),
                                    do_sample=True,   
                                    top_k=50, 
                                    max_length = 200,
                                    top_p=0.95, 
                                    num_return_sequences=1
                                )
            for i, sample_output in enumerate(sample_outputs):
                  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
            
            model.train()

        loss.backward()

        optimizer.step()

        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)       
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)
        
        with torch.no_grad():        

            outputs  = model(b_input_ids, 
#                            token_type_ids=None, 
                             attention_mask = b_masks,
                            labels=b_labels)
          
            loss = outputs[0]  
            
        batch_loss = loss.item()
        total_eval_loss += batch_loss        

    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    validation_time = format_time(time.time() - t0)    

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

## Saving the Fine-Tuned Model

In [ ]:
# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

output_dir = '/content/drive/MyDrive/Movie Script Generator/model_v2_save/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Good practice: save your training arguments together with the trained model
# torch.save(args, os.path.join(output_dir, 'training_args.bin'))

Saving model to /content/drive/MyDrive/Movie Script Generator/model_v2_save/


('/content/drive/MyDrive/Movie Script Generator/model_v2_save/tokenizer_config.json',
 '/content/drive/MyDrive/Movie Script Generator/model_v2_save/special_tokens_map.json',
 '/content/drive/MyDrive/Movie Script Generator/model_v2_save/vocab.json',
 '/content/drive/MyDrive/Movie Script Generator/model_v2_save/merges.txt',
 '/content/drive/MyDrive/Movie Script Generator/model_v2_save/added_tokens.json')

## Loading the Fine-Tuned Model

In [22]:
model = model.from_pretrained('/content/drive/MyDrive/Movie Script Generator/model_v2_save/')
model = model.to(device)

## Generate Text

The code below generates movie script lines based on genre prompts:

In [23]:
model.eval()

prompt = "COMEDY |"

generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)

print(generated)

sample_outputs = model.generate(
                                generated, 
                                #bos_token_id=random.randint(1,30000),
                                do_sample=True,   
                                top_k=50, 
                                max_length = 768,
                                top_p=0.95, 
                                num_return_sequences=10
                                )

for i, sample_output in enumerate(sample_outputs):
  print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[9858, 1961,   56,  930]], device='cuda:0')
0: COMEDY | <b> </b><b> INT. GROCER'S OFFICE - DAY </b><b> </b> There's a sign that says "Office Space For Rent". The place is in dire need of rent. <b> </b><b> </b><b> INT. LIVING ROOM - DAY </b><b> </b> The place is empty, only a few half-open bottles in one corner. Freddy's photo hangs on a wall. Next to it, the phone rings and rings. Freddy answers it. He smiles. <b> </b><b> FREDDY </b> Hello. <b> </b><b> KAREN </b> Is this you? <b> </b><b> FREDDY </b> It's me. I need an advance. <b> </b><b> CUT TO: </b><b> </b> Blade's footage of Karen screaming in agony. <b> </b><b> EXT. A DITCH - DAY </b><b> </b> Karen swims down the narrow drain. She passes an old ladder, and climbs on, looking around frantically for a way to get away. We see her POV through the crack. <b> </b><b> </b><b> INT. GROCER'S OFFICE - DAY </b><b> </b> Freddy is dialing a number on a phone. He holds up his hands. <b> </b><b> FREDDY (O.S.) </b> Karen, this is Karen Kin

In [24]:
model.eval()

prompt = "COMEDY |"

generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
generated = generated.to(device)

print(generated)

sample_outputs = model.generate(
                                generated, 
                                #bos_token_id=random.randint(1,30000),
                                do_sample=True,   
                                top_k=50, 
                                max_length = 768,
                                top_p=0.95, 
                                num_return_sequences=1
                                )

for i, sample_output in enumerate(sample_outputs):
  print("{}: {}\n\n".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


tensor([[9858, 1961,   56,  930]], device='cuda:0')
0: COMEDY | </b> I hate you, man. What does he say to you? <b> MR. SPOOK </b> He says he thinks you're... <b> CLIVE </b> A good one. But don't know why... (a beat) I know why. <b> MR. SPOOK </b> (after a beat) Is that right? <b> CLIVE </b> (smiles) I just don't... that. Mr. Sparkly gestures for her to put it in her hands. <b> MR. SPOOK </b> You'll help him to the door? <b> CLIVE </b> I don't know. <b> MR. SPOOK </b> Just then, something POUNDS on the door. <b> CLIVE </b> Who is it? <b> MR. SPOOK </b> (a beat) Man's inside. There's a beat. A knock. <b> CLIVE </b> Mr. Sparkly... <b> MR. SPOOK </b> (over his shoulder) Are you alright, Miss Vale? <b> CLIVE </b> Yes. <b> MR. SPOOK </b> Do you mind if I go inside? <b> CLIVE </b> Why don't you just wait outside? <b> MR. SPOOK </b> No, don't go. I can see you're upset. <b> CLIVE </b> Are you upset too? <b> MR. SPOOK </b> Yes, yes, I am. Now just just relax and I'll get out. <b> INT. ABANDONED

The cell below should generate movie scripts, taking into account previous context:

In [30]:
model.eval()

NUMBER_OF_HALF_MOVIE_SCRIPT_LINES_TO_GENERATE = 10

#prompt = "<|startoftext|>" # if I include this in the prompt, it generates garbage
genre_name = "COMEDY"
#prompt += " " + genre_name + " | "
prompt = genre_name + " | "

for i in range(0, NUMBER_OF_HALF_MOVIE_SCRIPT_LINES_TO_GENERATE):
  # debug prints
  #print("prompt:")
  #print(prompt)
  generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
  generated = generated.to(device)

  # debug prints
  #print(generated)

  sample_outputs = model.generate(
                                generated, 
                                #bos_token_id=random.randint(1,30000),
                                do_sample=True,   
                                top_k=50, 
                                max_length=768,
                                top_p=0.95, 
                                num_return_sequences=1
                                )

  # debug prints
  #print("sample_outputs:")
  #print(sample_outputs)
  #print("sample_outputs[0]:")
  #print(sample_outputs[0])
  sample_output_decoded = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
  sample_output_decoded_parts = sample_output_decoded.split(genre_name)
  # debug prints
  #print("sample_output_decoded_parts:")
  #print(sample_output_decoded_parts)
  #print("len(sample_output_decoded_parts):")
  #print(len(sample_output_decoded_parts))
  print(sample_output_decoded_parts[0] + genre_name + sample_output_decoded_parts[1])
  try:
    prompt = genre_name + sample_output_decoded_parts[2]
  except: # on this rare occasion, GPT-2 didn't generate the second genre_name in the middle of the generated sequence
    sample_output_decoded_tokens = sample_output_decoded.split(" ")
    sample_output_decoded_tokens_length = len(sample_output_decoded_tokens)
    half_sample_output_decoded_tokens_length = sample_output_decoded_tokens_length // 2
    prompt_after_genre_name_and_vertical_line = ""
    for token in sample_output_decoded_tokens[half_sample_output_decoded_tokens_length:sample_output_decoded_tokens_length]:
      prompt_after_genre_name_and_vertical_line += token + " "
    # debug prints
    #print("prompt_after_genre_name_and_vertical_line:")
    #print(prompt_after_genre_name_and_vertical_line)
    prompt = genre_name + " | " + prompt_after_genre_name_and_vertical_line

# print the last part of the last generated line
print(genre_name + sample_output_decoded_parts[2])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


COMEDY | urns are lit up. The house is empty except for two chairs with an antique piano, a pair of comfortable chairs and a small window. JACOB (v.o.) It occurred to me that I was having my last full word, since I couldn't begin to recall who I was. The light starts to come on. JACOB (v.o.) Then I woke up and there was a huge cloudburst of dust. I guess it wasn't just a cloudburst, but an amazing deluge. It was about a thousand gallons of pure oxygen. I had no clue who I was... and I had no idea who I was. We are close on JACOB. We see him for the first time. <b> JACOB (V.0.) </b> It's hard to tell whether I'm having an hallucination or just a hallucination. We are close on his eyes. <b> JACOB (V.O.) </b> If everything's all right, it might help to ask who I am. You see what I'm hallucinating about, on the other hand, have no idea. I can't tell. It's a huge, staggering, staggering dizzying sensation. <b> EXT. HAVENHURST - LATER </b> JACOB is sitting next to an open champagne bottle. H

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


COMEDY | am? And this woman I'm dying to marry? What is her name? A pretty middle aged woman in her forties. <b> JEZZIE </b> Elaine. <b> JACOB </b> Do you love her? <b> JEZZIE </b> Yes, but she's not even married. That's an important moment for me. <b> JACOB </b> What do you know about it? <b> JEZZIE </b> I just know, this is a very complicated world to me. <b> JACOB </b> I don't know what you mean. She's very young and you're a nice person and it's very interesting that you're married and together I'm very much alive. And I'll tell you why, because I'm attracted to you. In a way she's a love of my life. (Almost tenderly) Are we on the same page? <b> JEZZIE </b> Yes. <b> JACOB </b> And is it true about you? <b> JEZZIE </b> Yes, I'm sure it is. JACOB studies her a second. <b> JACOB </b> Are you happy? <b> JEZZIE </b> Yes, I am, but I'm not happy. <b> JACOB </b> You can't be happy? <b> JEZZIE </b> Yes. <b> JACOB </b> Why? <b> JEZZIE </b> I don't know what to think. <b> JACOB </b> Why? <b

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

COMEDY | <b> JACOB </b> Why? <b> JEZZIE </b> Because you're so beautiful. She's so good to me. I'm not really her lover. JACOB stares at her a long time. She still doesn't answer. <b> JACOB </b> Are you sure? <b> JEZZIE </b> Yes. <b> JACOB </b> So, what does it say? <b> JEZZIE </b> I don't know. <b> JACOB </b> What do you mean? <b> JEZZIE </b> That it's too much. <b> JACOB </b> Oh, great. That's great. <b> JEZZIE </b> Yeah. <b> JACOB </b> Great. Okay. Let's go see a movie. <b> JEZZIE </b> Yeah? <b> JACOB </b> Yeah. <b> JEZZIE </b> Can we see it? <b> JACOB </b> Yeah. <b> JEZZIE </b> Why? <b> JACOB </b> You mean, what? <b> JEZZIE </b> You mean, tell me. I want to see it. <b> JACOB </b> Yes. Of course. Okay. <b> JEZZIE </b> Wait a second. What does it say? <b> JACOB </b> (very quiet) It says, "Please tell me I'm beautiful." <b> JEZZIE </b> Are you happy? <b> JACOB </b> Yes. Happy, yes. Happy. Happy. <b> JEZZIE </b> Say, is that a pretty thing? <b> JACOB </b> Yes. Say, are you happy? JEZZI

IndexError: ignored

The above code doesn't produce good results most of the time (in 7/10 runs it produced garbage).

Let's try generating a movie script with a prompt that looks as follows:

**half of the generated line | genre name**

In [51]:
model.eval()

NUMBER_OF_HALF_MOVIE_SCRIPT_LINES_TO_GENERATE = 10

genre_name = "COMEDY"
prompt = genre_name + " | "

for i in range(0, NUMBER_OF_HALF_MOVIE_SCRIPT_LINES_TO_GENERATE):
  # debug prints below
  #print("prompt:")
  #print(prompt)
  generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
  generated = generated.to(device)

  # debug prints below
  #print(generated)

  sample_outputs = model.generate(
                                generated, 
                                #bos_token_id=random.randint(1,30000),
                                do_sample=True,   
                                top_k=50, 
                                max_length=768,
                                top_p=0.95, 
                                num_return_sequences=1
                                )

  sample_output_decoded = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
  # debug prints below
  #print("sample_output_decoded:")
  #print(sample_output_decoded)
  sample_output_decoded_parts = sample_output_decoded.split(genre_name)
  # debug prints below
  #print("sample_output_decoded_parts:")
  #print(sample_output_decoded_parts)
  #print("len(sample_output_decoded_parts):")
  #print(len(sample_output_decoded_parts))
  prompt = sample_output_decoded_parts[1][2:] + " " + genre_name + " | " # the sample_output_decoded_parts[1][2:] gets rid
                                                                         # of the " | "

  print(sample_output_decoded_parts[0] + genre_name + sample_output_decoded_parts[1])
  
  if (len(sample_output_decoded_parts) == 2): # the midway genre tag wasn't generated
    sample_output_decoded_tokens = sample_output_decoded.split(" ")
    sample_output_decoded_tokens_length = len(sample_output_decoded_tokens)
    half_sample_output_decoded_tokens_length = sample_output_decoded_tokens_length // 2
    prompt_before_genre_name_and_vertical_line = ""
    for token in sample_output_decoded_tokens[:half_sample_output_decoded_tokens_length]:
      prompt_before_genre_name_and_vertical_line += token + " "
    # debug prints below
    #print("prompt_before_genre_name_and_vertical_line:")
    #print(prompt_before_genre_name_and_vertical_line)
    if (prompt_before_genre_name_and_vertical_line != genre_name + " | "):
      prompt = prompt_before_genre_name_and_vertical_line + genre_name + " | " # avoids duplicating the genre tag and the vertical line
    else:
      prompt = genre_name + " | "

# print the last part of the last generated line
print(genre_name + " | " + sample_output_decoded_parts[2])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


COMEDY | ier. <b> GITTES </b> Well, I guess you do. <b> DENNIS </b> I'm glad you're happy about this, Harry. That puts her to work. <b> GITTES </b> You're a good man. <b> (THEN) </b> It's a good job. <b> EXT. MULECY </b> As a limousine starts to roll down the mountain road, Gittes and Burke emerge on the street flanked by FBI Agents. Burke has an FBI BADGE pinned to his back, identifying the man as FBI Agent Gittes. <b> BURKE </b> Gittes. <b> GITTES </b> So what the hell's he doin' with this guy now? <b> BURKE </b> You don't know that guy. He used to live in the city. The guy's the real deal. He always had a weakness for special- forces work. (shaking Burke's arm) How about you take this guy and see how he reacts. <b> GITTES </b> Burke's a good man. <b> BURKE </b> And you know what I'm sayin'? 'Yeah, you got a lot on him, too.' Gittes is taken aback by Burke's tough exterior. <b> INT. TOWNHOUSE - OLD TIMER TAVERN </b> Burke and Gittes enter the empty diner to find Burke and Gittes, dre

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 ier. <b> GITTES </b> Well, I guess you do. <b> DENNIS </b> I'm glad you're happy about this, Harry. That puts her to work. <b> GITTES </b> You're a good man. <b> (THEN) </b> It's a good job. <b> EXT. MULECY </b> As a limousine starts to roll down the mountain road, Gittes and Burke emerge on the street flanked by FBI Agents. Burke has an FBI BADGE pinned to his back, identifying the man as FBI Agent Gittes. <b> BURKE </b> Gittes. <b> GITTES </b> So what the hell's he doin' with this guy now? <b> BURKE </b> You don't know that guy. He used to live in the city. The guy's the real deal. He always had a weakness for special- forces work. (shaking Burke's arm) How about you take this guy and see how he reacts. <b> GITTES </b> Burke's a good man. <b> BURKE </b> And you know what I'm sayin'? 'Yeah, you got a lot on him, too.' Gittes is taken aback by Burke's tough exterior. <b> INT. TOWNHOUSE - OLD TIMER TAVERN </b> Burke and Gittes enter the empty diner to find Burke and Gittes, dressed in 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

 icles to this place? They don't even have a TV or a computer! <b> LOU </b> I didn't think it was that big to come up and do a hik-a with my stuff. <b> TED </b> We'll have to give it a little try. <b> CUT TO: </b> <b> INT. ANOTHER PART OF THE PARK - NIGHT </b> <b> TED </b> You know, I've been thinking about my wife. What if I made this whole thing about her? If I told her I wanted to get married or something. I wouldn't want to let her go. You know what I mean? I wouldn't take it, not unless I knew the answer. But then I was wondering if maybe we could go into my own private world where she'd be able to accept that I'd be good at it. <b> LOU </b> (excited) Then what? <b> TED </b> Then I could tell her it was hopeless. That I'm not going to marry her. She might be unhappy. Maybe she'll have tears in her eyes. <b> LOU </b> I'm not gonna marry her. I want to be the father. <b> TED </b> Me neither. If I'm gonna be the father, I need someone like you to take charge. I need someone with some

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 icles to this place? They don't even have a TV or a computer! <b> LOU </b> I didn't think it was that big to come up and do a hik-a with my stuff. <b> TED </b> We'll have to give it a little try. <b> CUT TO: </b> <b> INT. ANOTHER PART OF THE PARK - NIGHT </b> <b> TED </b> You know, I've been thinking about my wife. What if I made this whole thing about her? If I told her I wanted to get married or something. I wouldn't want to let her go. You know what I mean? I wouldn't take it, not unless I knew the answer. But then I was wondering if maybe we could go into my own private world where she'd be able to accept that I'd be good at it. <b> LOU </b> (excited) Then what? <b> TED </b> Then I could tell her it was hopeless. That I'm not going to marry her. She might be unhappy. Maybe she'll have tears in her eyes. <b> LOU </b> I'm not gonna marry her. I want to be the father. <b> TED </b> Me neither. If I'm gonna be the father, I need someone like you to take charge. I need someone with some

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 ike. I would do it. <b> TED </b> You're probably right. We could probably settle up together after work. <b> LOU </b> You mean after work? <b> TED </b> That might be a possibility. <b> LOU </b> I'm very, very happy with this idea. <b> TED </b> With a million dollars?? <b> CUT TO: </b> <b> EXT. THE COUNCIL BUILDING - LATER </b> The whole park is lit up with bright, colorful lights and a huge, banner reading: CITY <b> COUNCIL - BALTIMORE </b> <b> A NEW ERA AGENCY </b> <b> A NEW ERA AGENCY </b> <b> A NEW ERA AGENCY </b> <b> A NEW ERA AGENCY </b> <b> A NEW ERA AGENCY </b> <b> A NEW ERA AGENCY </b> The NEW ERA AGENCY is one of several offices that have been converted into WAREHOUSE MARKETS and CUBICLES. Some are huge and have long lines of CUSTOMERS waiting to be purchased, most are small, SCARY SERVICE CRATES, others are overflowing with junk, etc. All of  COMEDY | ills is a series of TORTURES wrapped in plastic bags, and sold. <b> ANDREW WALSH </b> They work as an arm's length store. <b>

### Note: The above code very often generates movie dialogue in French... I don't know why

### Note: Sometimes the code above generates a different genre name than the one in the prompt

### Note: Sometimes the code above generates 2 genre names such that the second genre tag is near the end or the beginning of the sequence (and not halfway)

### Note: The above code generates good, but repetitive output about 40% of the time (I ran it ~10 times)

Let's try generating with the sequence that comes after the midway genre tag (without the midway genre tag):

In [56]:
model.eval()

NUMBER_OF_HALF_MOVIE_SCRIPT_LINES_TO_GENERATE = 10

genre_name = "COMEDY"
prompt = genre_name + " | "

for i in range(0, NUMBER_OF_HALF_MOVIE_SCRIPT_LINES_TO_GENERATE):
  print("prompt:")
  print(prompt)
  generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
  generated = generated.to(device)

  #print(generated)

  sample_outputs = model.generate(
                                generated, 
                                #bos_token_id=random.randint(1,30000),
                                do_sample=True,   
                                top_k=50, 
                                max_length=768,
                                top_p=0.95, 
                                num_return_sequences=1
                                )

  #print("sample_outputs:")
  #print(sample_outputs)
  #print("sample_outputs[0]:")
  #print(sample_outputs[0])
  sample_output_decoded = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
  sample_output_decoded_parts = sample_output_decoded.split(genre_name)
  print("len(sample_output_decoded_parts):")
  print(len(sample_output_decoded_parts))
  print(sample_output_decoded_parts[0] + genre_name + sample_output_decoded_parts[1])
  try:
    prompt = sample_output_decoded_parts[2][3:] # gets rid of the " | "
  except: # on this rare occasion, GPT-2 didn't generate the second genre_name in the middle of the generated sequence
    sample_output_decoded_tokens = sample_output_decoded.split(" ")
    sample_output_decoded_tokens_length = len(sample_output_decoded_tokens)
    half_sample_output_decoded_tokens_length = sample_output_decoded_tokens_length // 2
    prompt_after_genre_name_and_vertical_line = ""
    for token in sample_output_decoded_tokens[half_sample_output_decoded_tokens_length:sample_output_decoded_tokens_length]:
      prompt_after_genre_name_and_vertical_line += token + " "
    prompt = prompt_after_genre_name_and_vertical_line

# print the last part of the last generated line
print(genre_name + sample_output_decoded_parts[2])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


prompt:
COMEDY | 


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


len(sample_output_decoded_parts):
2
COMEDY | ieta il est a peut encore! Ils se retournent au chien � sept d'une grande musique de la vitrum. <b> LORI </b> Monsieur Tonton, o� est le regard! <b> TONNO </b> Voil�, ma petite j�ureur. <b> LORI </b> En faisait des deux pour voyons. Une autre fois est tomb�e ses bougies pour les choses. Tonton enjoint Gabriel qui se retourne vers le haut de la musique de ses draps. Gabriel se retourne vers le haut de la musique, mais vu leurs en faisait des poches. <b> GABRIEL FOUQUET </b> Monsieur Gabriel est inscrit : Gabriel et les retrouves des ducs bougies, et que le autre fois descend du mur, on parlant le haut de sa musique. <b> LORI </b> Faut tout de nous, monsieur, qui �tes l'argenter au milieu de la musique. Il se retourne vers Gabriel, qui est assis au milieu de la musique, et regarde Gabriel. Lorsl�e sort un bonbon dans le mur. <b> LORI </b> Pardonnez-moi, Monsieur Tonton, vous avez le mauvaire, la musique est inscrit : Gabriel. <b> GABRIEL FOUQU

IndexError: ignored

### Note: The above code crashed in all 5 times I ran it since **sample_output_decoded_parts** had the length of 1. This isn't expected behaviour, so I'm moving on from this code.

### Let me try different k values:

Let's try **top_k = 10**:

In [60]:
model.eval()

NUMBER_OF_HALF_MOVIE_SCRIPT_LINES_TO_GENERATE = 10

#prompt = "<|startoftext|>" # if I include this in the prompt, it generates garbage
genre_name = "COMEDY"
#prompt += " " + genre_name + " | "
prompt = genre_name + " | "

for i in range(0, NUMBER_OF_HALF_MOVIE_SCRIPT_LINES_TO_GENERATE):
  # debug prints
  #print("prompt:")
  #print(prompt)
  generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
  generated = generated.to(device)

  # debug prints
  #print(generated)

  sample_outputs = model.generate(
                                generated, 
                                #bos_token_id=random.randint(1,30000),
                                do_sample=True,   
                                top_k=10, 
                                max_length=768,
                                top_p=0.95, 
                                num_return_sequences=1
                                )

  # debug prints
  #print("sample_outputs:")
  #print(sample_outputs)
  #print("sample_outputs[0]:")
  #print(sample_outputs[0])
  sample_output_decoded = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
  sample_output_decoded_parts = sample_output_decoded.split(genre_name)
  # debug prints
  #print("sample_output_decoded_parts:")
  #print(sample_output_decoded_parts)
  #print("len(sample_output_decoded_parts):")
  #print(len(sample_output_decoded_parts))
  print(sample_output_decoded_parts[0] + genre_name + sample_output_decoded_parts[1])
  try:
    prompt = genre_name + sample_output_decoded_parts[2]
  except: # on this rare occasion, GPT-2 didn't generate the second genre_name in the middle of the generated sequence
    sample_output_decoded_tokens = sample_output_decoded.split(" ")
    sample_output_decoded_tokens_length = len(sample_output_decoded_tokens)
    half_sample_output_decoded_tokens_length = sample_output_decoded_tokens_length // 2
    prompt_after_genre_name_and_vertical_line = ""
    for token in sample_output_decoded_tokens[half_sample_output_decoded_tokens_length:sample_output_decoded_tokens_length]:
      prompt_after_genre_name_and_vertical_line += token + " "
    # debug prints
    #print("prompt_after_genre_name_and_vertical_line:")
    #print(prompt_after_genre_name_and_vertical_line)
    prompt = genre_name + " | " + prompt_after_genre_name_and_vertical_line

# print the last part of the last generated line
print(genre_name + sample_output_decoded_parts[2])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

COMEDY | izhts. Asael suddenly looks up. <b> ASAEL </b> They have the gates. They've got the Ark. They've got the Ark. <b> GABRIEL </b> But they have no way! <b> (CONTINUED) </b> <b> </b> <b> </b> <b> </b> <b> </b> SALMON Revision - 8-18-07 71A. <b> 73 CONTINUED: 73 </b> <b> ASAEL </b> No they don't! It's too dangerous. <b> GABRIEL </b> They must have it. The gates have to be locked. <b> ASAEL </b> No they won't! They can't! <b> GABRIEL </b> They've got it, they must have it! <b> (CONTINUED) </b> <b> </b> <b> </b> <b> </b> <b> </b> SALMON Revision - 8-18-07 71B. <b> 73 CONTINUED: (2) 73 </b> <b> GABRIEL </b> <b> BUT THEY GOT THE KORFMAN'S KEYS! </b> <b> ASAEL </b> <b> THEY GOT THE KORFMAN'S KEYS! </b> <b> GABRIEL </b> <b> THEY GOT THE KORFMAN'S KEYS! </b> <b> ASAEL </b> They're locked. <b> GABRIEL </b> Locked! <b> ASAEL </b> They've got the gates, the Ark. The people are coming! <b> GABRIEL </b> <b> THEY'LL GIVE IT UP OR THEY'LL GET </b> <b> KILL US ALL! </b> <b> RESCUE </b> <b> GABRIE

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


COMEDY |      
COMEDY |     


IndexError: ignored

Let's try **top_k = 100**:

In [68]:
model.eval()

NUMBER_OF_HALF_MOVIE_SCRIPT_LINES_TO_GENERATE = 10

#prompt = "<|startoftext|>" # if I include this in the prompt, it generates garbage
genre_name = "COMEDY"
#prompt += " " + genre_name + " | "
prompt = genre_name + " | "

for i in range(0, NUMBER_OF_HALF_MOVIE_SCRIPT_LINES_TO_GENERATE):
  # debug prints
  #print("prompt:")
  #print(prompt)
  generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
  generated = generated.to(device)

  # debug prints
  #print(generated)

  sample_outputs = model.generate(
                                generated, 
                                #bos_token_id=random.randint(1,30000),
                                do_sample=True,   
                                top_k=100, 
                                max_length=768,
                                top_p=0.95, 
                                num_return_sequences=1
                                )

  # debug prints
  #print("sample_outputs:")
  #print(sample_outputs)
  #print("sample_outputs[0]:")
  #print(sample_outputs[0])
  sample_output_decoded = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
  sample_output_decoded_parts = sample_output_decoded.split(genre_name)
  # debug prints
  #print("sample_output_decoded_parts:")
  #print(sample_output_decoded_parts)
  #print("len(sample_output_decoded_parts):")
  #print(len(sample_output_decoded_parts))
  print(sample_output_decoded_parts[0] + genre_name + sample_output_decoded_parts[1])
  try:
    prompt = genre_name + sample_output_decoded_parts[2]
  except: # on this rare occasion, GPT-2 didn't generate the second genre_name in the middle of the generated sequence
    sample_output_decoded_tokens = sample_output_decoded.split(" ")
    sample_output_decoded_tokens_length = len(sample_output_decoded_tokens)
    half_sample_output_decoded_tokens_length = sample_output_decoded_tokens_length // 2
    prompt_after_genre_name_and_vertical_line = ""
    for token in sample_output_decoded_tokens[half_sample_output_decoded_tokens_length:sample_output_decoded_tokens_length]:
      prompt_after_genre_name_and_vertical_line += token + " "
    # debug prints
    #print("prompt_after_genre_name_and_vertical_line:")
    #print(prompt_after_genre_name_and_vertical_line)
    prompt = genre_name + " | " + prompt_after_genre_name_and_vertical_line

# print the last part of the last generated line
print(genre_name + sample_output_decoded_parts[2])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


COMEDY | irectly in front of him is a small, black glass apparatus. It is a machine designed to control the movement of the water at liquid-level. This device is known as a "water port." There are hundreds of sensors that are wired to it, operated by a small, wired-to-the- environment machine. Once the control is complete, the water port slowly lowers, slowly lowering. As the pressure increases, the glass sphere will slowly lower, until it is level with the crystalline floor. This is known as the "V." <b> SPHINX </b> (in awe) This is the real thing... <b> ZAPHOD </b> Yes, it is... The ship is about ten meters wide. Half the atmosphere in the room will be contained inside a glass sphere. All of this will in turn produce tremendous heat and tremendous electromagnetic pulse that will spread throughout the ship. The glass sphere will slow at about 5 meters per second. Splashing is impossible, even impossible in an underground sea. Suddenly, in the sphere itself, a huge mass of blue-white L

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


COMEDY | standing around their stations, and everyone is in a state of shock. <b> SULU </b> No one move! A collective breath of relief goes through the ship. <b> SCIMP </b> They're okay. It's a miracle. <b> PICARD </b> It's great to be safe here... <b> KLUTE </b> Let's get the hell out of here. <b> PICARD </b> I'm a captain. If we're going to stay this ship, we need to keep moving. <b> KIRK </b> Sir, we have no time to argue. All of a sudden the ship's engines go out in a series of sputters, then stalls. <b> JIMMY PIG </b> They'll stop! All engines go out! Jimmy Pig turns away in disgust. <b> KIRK </b> Aw, crap. I didn't even know that ship had half a minute to stop. (to Kirk) Jim, you okay? <b> KIRK </b> I'm all right. <b> JIMMY PIG </b> Can't you get past the fact that there's a giant, silent ship in the sky that's going to die? Everyone stops what they are doing. <b> HECK </b> We're not going to die here. Kirk lets this sink in. <b> KIRK </b> You mean you want to tell us why? <b> HE

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

COMEDY | <b> EXT. THE HOUSE - DAY </b> Through the front door, in the dim light of dawn, we see the front porch. Kirk steps into the shadows. <b> CUT TO: </b> <b> INT. BEDROOM - DAY </b> A small bedroom off one of the fifties. Kirk paces inside in his robe, as if he doesn't know what to say. The house is a mess. It's been stripped of his personal possessions. Stuffed posters, newspaper clippings, old photographs, old clothes, old shoes. Kirk just stands there a long beat, staring at nothing. <b> NICOLE (O.S.) </b> And? Kirk snaps the door shut. NICOLE has come in, with groceries and an umbrella from the kitchen. She stands beside the door, but he looks left and right, making sure she's there. <b> NICOLE </b> Mr. Franson, I know the feeling you left before we met. That you... <b> KIRK </b> I wouldn't trade his shoes for your socks! And I'd be very nervous if I told you I liked 'em. <b> NICOLE </b> Of course you would. <b> KIRK </b> This is your house. <b> NICOLE </b> I remember it well.

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


COMEDY | financial side... He finally gets up from the bed. <b> NICOLE </b> Listen, I was just going to say -- how would you feel if you actually knew that your son was still alive? Kirk nods, agreeing. <b> NICOLE </b> I'm sorry to tell you that it is a long story. And I don't want you to tell me where I am. This is my story now. I've got a long way to go. She gets into the car and starts the engine. As they drive off, we can see inside the car the small man, RONSON, has been watching them get out. <b> EXT. STREET - DAY </b> They come to a stop. <b> NICOLE </b> But, Kirk, isn't it interesting...? What's the point. If this stuff gets to the bottom, I'll never be able to be a father again. (beat) But, Kirk... what's wrong with you? <b> KIRK </b> I don't know. (beat) Maybe I don't need a father anymore. They both look at the street, all expressionless. Then: <b> NICOLE </b> So, you don't expect me to have a heart right now? The two share a look. It almost looks like a kiss. The car pulls 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


COMEDY | <b> INT. KIRK'S KITCHEN - DAY </b> Kirk is in the kitchen doing dinner. In the background is Nina. Suddenly, his hand drags something from the back. The fingers slip. <b> NINA </b> Oh my God! Is that your son? A smile. He looks at her, a little confused. Nina pinches his cheek. <b> NINA </b> I didn't mean that. I didn't mean that. I just meant that if someone like that happens to me, I can't ever let him do that again. I'll lose it. <b> KIRK </b> What have I done to your son? <b> NINA </b> Nothing. Just nothing. Now, you just go away and get some sleep. Maybe that's all you have left. I'll leave. <b> KIRK </b> We've got to go. <b> NINA </b> Will you just listen to me? This isn't going to be easy. It's not going to be easy. <b> KIRK </b> I know. <b> NINA </b> Look, you look great -- I know you look great. But if you want to have a good time, then let's go get some sleep. Do you want a beer or something? <b> KIRK </b> I don't know what to tell them. <b> INT. UPSTAIRS HALLWAY - D

### Note: The above model generated garbage outputs about 3/8 of the times I ran it, it generated repetitive output 2/8 times I ran it, but on the other 3/8 times it generated solid movie scripts.

Let's try **top_k = 5**:

In [72]:
model.eval()

NUMBER_OF_HALF_MOVIE_SCRIPT_LINES_TO_GENERATE = 10

#prompt = "<|startoftext|>" # if I include this in the prompt, it generates garbage
genre_name = "COMEDY"
#prompt += " " + genre_name + " | "
prompt = genre_name + " | "

for i in range(0, NUMBER_OF_HALF_MOVIE_SCRIPT_LINES_TO_GENERATE):
  # debug prints
  #print("prompt:")
  #print(prompt)
  generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
  generated = generated.to(device)

  # debug prints
  #print(generated)

  sample_outputs = model.generate(
                                generated, 
                                #bos_token_id=random.randint(1,30000),
                                do_sample=True,   
                                top_k=5, 
                                max_length=768,
                                top_p=0.95, 
                                num_return_sequences=1
                                )

  # debug prints
  #print("sample_outputs:")
  #print(sample_outputs)
  #print("sample_outputs[0]:")
  #print(sample_outputs[0])
  sample_output_decoded = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
  sample_output_decoded_parts = sample_output_decoded.split(genre_name)
  # debug prints
  #print("sample_output_decoded_parts:")
  #print(sample_output_decoded_parts)
  #print("len(sample_output_decoded_parts):")
  #print(len(sample_output_decoded_parts))
  print(sample_output_decoded_parts[0] + genre_name + sample_output_decoded_parts[1])
  try:
    prompt = genre_name + sample_output_decoded_parts[2]
  except: # on this rare occasion, GPT-2 didn't generate the second genre_name in the middle of the generated sequence
    sample_output_decoded_tokens = sample_output_decoded.split(" ")
    sample_output_decoded_tokens_length = len(sample_output_decoded_tokens)
    half_sample_output_decoded_tokens_length = sample_output_decoded_tokens_length // 2
    prompt_after_genre_name_and_vertical_line = ""
    for token in sample_output_decoded_tokens[half_sample_output_decoded_tokens_length:sample_output_decoded_tokens_length]:
      prompt_after_genre_name_and_vertical_line += token + " "
    # debug prints
    #print("prompt_after_genre_name_and_vertical_line:")
    #print(prompt_after_genre_name_and_vertical_line)
    prompt = genre_name + " | " + prompt_after_genre_name_and_vertical_line

# print the last part of the last generated line
print(genre_name + sample_output_decoded_parts[2])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

COMEDY |  
COMEDY |   
COMEDY |    
COMEDY |    
COMEDY |    
COMEDY |    
COMEDY |    
COMEDY |    
COMEDY |    
COMEDY |    


IndexError: ignored

### Note: the above approach generated garbage outputs most of the time (I ran it about 4 times)

Let's try **top_k = 1**:

In [75]:
model.eval()

NUMBER_OF_HALF_MOVIE_SCRIPT_LINES_TO_GENERATE = 10

#prompt = "<|startoftext|>" # if I include this in the prompt, it generates garbage
genre_name = "COMEDY"
#prompt += " " + genre_name + " | "
prompt = genre_name + " | "

for i in range(0, NUMBER_OF_HALF_MOVIE_SCRIPT_LINES_TO_GENERATE):
  # debug prints
  #print("prompt:")
  #print(prompt)
  generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
  generated = generated.to(device)

  # debug prints
  #print(generated)

  sample_outputs = model.generate(
                                generated, 
                                #bos_token_id=random.randint(1,30000),
                                do_sample=True,   
                                top_k=1, 
                                max_length=768,
                                top_p=0.95, 
                                num_return_sequences=1
                                )

  # debug prints
  #print("sample_outputs:")
  #print(sample_outputs)
  #print("sample_outputs[0]:")
  #print(sample_outputs[0])
  sample_output_decoded = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
  sample_output_decoded_parts = sample_output_decoded.split(genre_name)
  # debug prints
  #print("sample_output_decoded_parts:")
  #print(sample_output_decoded_parts)
  #print("len(sample_output_decoded_parts):")
  #print(len(sample_output_decoded_parts))
  print(sample_output_decoded_parts[0] + genre_name + sample_output_decoded_parts[1])
  try:
    prompt = genre_name + sample_output_decoded_parts[2]
  except: # on this rare occasion, GPT-2 didn't generate the second genre_name in the middle of the generated sequence
    sample_output_decoded_tokens = sample_output_decoded.split(" ")
    sample_output_decoded_tokens_length = len(sample_output_decoded_tokens)
    half_sample_output_decoded_tokens_length = sample_output_decoded_tokens_length // 2
    prompt_after_genre_name_and_vertical_line = ""
    for token in sample_output_decoded_tokens[half_sample_output_decoded_tokens_length:sample_output_decoded_tokens_length]:
      prompt_after_genre_name_and_vertical_line += token + " "
    # debug prints
    #print("prompt_after_genre_name_and_vertical_line:")
    #print(prompt_after_genre_name_and_vertical_line)
    prompt = genre_name + " | " + prompt_after_genre_name_and_vertical_line

# print the last part of the last generated line
print(genre_name + sample_output_decoded_parts[2])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

COMEDY |  
COMEDY |   
COMEDY |    
COMEDY |    
COMEDY |    
COMEDY |    
COMEDY |    
COMEDY |    
COMEDY |    
COMEDY |    


IndexError: ignored

### Note: the above approach generated garbage outputs all of the time (I ran it 3 times)

Let's try a different prompt structure (**half of the generated movie line | genre name**) with **top_k = 100**:

In [78]:
model.eval()

NUMBER_OF_HALF_MOVIE_SCRIPT_LINES_TO_GENERATE = 10

genre_name = "COMEDY"
prompt = genre_name + " | "

for i in range(0, NUMBER_OF_HALF_MOVIE_SCRIPT_LINES_TO_GENERATE):
  # debug prints below
  #print("prompt:")
  #print(prompt)
  generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
  generated = generated.to(device)

  # debug prints below
  #print(generated)

  sample_outputs = model.generate(
                                generated, 
                                #bos_token_id=random.randint(1,30000),
                                do_sample=True,   
                                top_k=100, 
                                max_length=768,
                                top_p=0.95, 
                                num_return_sequences=1
                                )

  sample_output_decoded = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
  # debug prints below
  #print("sample_output_decoded:")
  #print(sample_output_decoded)
  sample_output_decoded_parts = sample_output_decoded.split(genre_name)
  # debug prints below
  #print("sample_output_decoded_parts:")
  #print(sample_output_decoded_parts)
  #print("len(sample_output_decoded_parts):")
  #print(len(sample_output_decoded_parts))
  prompt = sample_output_decoded_parts[1][2:] + " " + genre_name + " | " # the sample_output_decoded_parts[1][2:] gets rid
                                                                         # of the " | "

  print(sample_output_decoded_parts[0] + genre_name + sample_output_decoded_parts[1])
  
  if (len(sample_output_decoded_parts) == 2): # the midway genre tag wasn't generated
    sample_output_decoded_tokens = sample_output_decoded.split(" ")
    sample_output_decoded_tokens_length = len(sample_output_decoded_tokens)
    half_sample_output_decoded_tokens_length = sample_output_decoded_tokens_length // 2
    prompt_before_genre_name_and_vertical_line = ""
    for token in sample_output_decoded_tokens[:half_sample_output_decoded_tokens_length]:
      prompt_before_genre_name_and_vertical_line += token + " "
    # debug prints below
    #print("prompt_before_genre_name_and_vertical_line:")
    #print(prompt_before_genre_name_and_vertical_line)
    if (prompt_before_genre_name_and_vertical_line != genre_name + " | "):
      prompt = prompt_before_genre_name_and_vertical_line + genre_name + " | " # avoids duplicating the genre tag and the vertical line
    else:
      prompt = genre_name + " | "

# print the last part of the last generated line
print(genre_name + " | " + sample_output_decoded_parts[2])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


COMEDY | 


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


COMEDY 
 COMEDY |  


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

????? What kind of website are you? <b> DILLON </b> Oh. Um, uh, I'd like to say that one of my favorite games is called Monkey over: Do you have your phone or phone? <b> TERRY </b> Ok, so I've decided to throw away my phone and go to work. And it's been a while and I thought I'd just hang up and let the guys devour this new thing and go at it from the first minute it's gonna start. I mean, I like the story a lot. It's like, a little bit different from the other games. <b> TERRY </b> Oh? <b> DILLON </b> You know what? I've never played a video game in my whole life. It takes a really long time. And when you play it, you know, you know, like a hundred times. And I've got to make it really painful for my friends. I don't have a lot of friends. Terry looks stunned. <b> TERRY </b> I love you. <b> </b> <b> </b> <b> </b> <b> </b> <b> 79. </b> <b> TERRY/DILLON </b> It's ok. I like you. But, uh... there's just so many feelings. <b> DILLON COMEDY | iewu> Yeah? I've been so lonely and out lately,

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


????? What kind of website are you? <b> DILLON </b> Oh. Um, uh, I'd like to say that one of my favorite games is called Monkey over: Do you have your phone or phone? <b> TERRY </b> Ok, so I've decided to throw away my phone and go to work. And it's been a while and I thought I'd just hang up and let the guys devour this new thing and go at it from the first minute it's gonna start. I mean, I like the story a lot. It's like, a little bit different from the other games. <b> TERRY </b> Oh? <b> DILLON </b> You know what? I've never played a video game in my whole life. It takes a really long time. And when you play it, you know, you know, like a hundred times. And I've got to make it really painful for my friends. I don't have a lot of friends. Terry looks stunned. <b> TERRY </b> I love you. <b> </b> <b> </b> <b> </b> <b> </b> <b> 79. </b> <b> TERRY/DILLON </b> It's ok. I like you. But, uh... there's just so many feelings. <b> DILLON COMEDY | iewu> Yeah? I've been so lonely and out lately,

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 iewu> Yeah? I've been so lonely and out lately, so I'm really happy to have you over to spend time with me in my apartment. <b> TERRY </b> Yeah, I've always wanted to talk to you but I really think this is kind of a good time to  COMEDY | �� you know, be happy because I know I already have an apartment of my own. And I also like you, being there when I have a drink at six. So I'm really gonna write this down for you tomorrow night. Can I ever write my name down? <b> LILA </b> I think so, so. Terry sits up, glancing at the calendar. <b> TERRY </b> Do you know what time it is? <b> LILA </b> Actually, it's four-thirty. And you're gonna be late today. Just then, a very curious Maid comes in, walks over to Terry and kisses him on the cheek. <b> MAID </b> Hi! Terry, you ready for the present? <b> TERRY </b> Sure, I am. <b> MAID </b> Great meeting you. The Maid and the Maid walk on, as Terry walks back into his apartment. <b> CUT TO: </b> <b> THE MAID </b> I'm so happy we both have dinner. <

### Note: The above code generates repetitive output (tested ~4 times)

Let's try **top_k = 10**:

In [81]:
model.eval()

NUMBER_OF_HALF_MOVIE_SCRIPT_LINES_TO_GENERATE = 10

genre_name = "COMEDY"
prompt = genre_name + " | "

for i in range(0, NUMBER_OF_HALF_MOVIE_SCRIPT_LINES_TO_GENERATE):
  # debug prints below
  #print("prompt:")
  #print(prompt)
  generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
  generated = generated.to(device)

  # debug prints below
  #print(generated)

  sample_outputs = model.generate(
                                generated, 
                                #bos_token_id=random.randint(1,30000),
                                do_sample=True,   
                                top_k=10, 
                                max_length=768,
                                top_p=0.95, 
                                num_return_sequences=1
                                )

  sample_output_decoded = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
  # debug prints below
  #print("sample_output_decoded:")
  #print(sample_output_decoded)
  sample_output_decoded_parts = sample_output_decoded.split(genre_name)
  # debug prints below
  #print("sample_output_decoded_parts:")
  #print(sample_output_decoded_parts)
  #print("len(sample_output_decoded_parts):")
  #print(len(sample_output_decoded_parts))
  prompt = sample_output_decoded_parts[1][2:] + " " + genre_name + " | " # the sample_output_decoded_parts[1][2:] gets rid
                                                                         # of the " | "

  print(sample_output_decoded_parts[0] + genre_name + sample_output_decoded_parts[1])
  
  if (len(sample_output_decoded_parts) == 2): # the midway genre tag wasn't generated
    sample_output_decoded_tokens = sample_output_decoded.split(" ")
    sample_output_decoded_tokens_length = len(sample_output_decoded_tokens)
    half_sample_output_decoded_tokens_length = sample_output_decoded_tokens_length // 2
    prompt_before_genre_name_and_vertical_line = ""
    for token in sample_output_decoded_tokens[:half_sample_output_decoded_tokens_length]:
      prompt_before_genre_name_and_vertical_line += token + " "
    # debug prints below
    #print("prompt_before_genre_name_and_vertical_line:")
    #print(prompt_before_genre_name_and_vertical_line)
    if (prompt_before_genre_name_and_vertical_line != genre_name + " | "):
      prompt = prompt_before_genre_name_and_vertical_line + genre_name + " | " # avoids duplicating the genre tag and the vertical line
    else:
      prompt = genre_name + " | "

# print the last part of the last generated line
print(genre_name + " | " + sample_output_decoded_parts[2])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

COMEDY | 
COMEDY 
 COMEDY |  


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

 COMEDY 
 COMEDY |  
 COMEDY 
 COMEDY |  
 COMEDY 


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 COMEDY | iz, it's not the same, but it's the same. And you're a smart man, and you're a smart person, and you're a smart person is... and I am. And that's exactly what I am, because I am the smartest person I've ever met, because I know where I want to go someday. But I'm not going to be a smart person. I'm going to be a jerk. And I'm not going to be a jerk. I'm not going to be a jerk. You're not going to be a jerk. I am a jerk. And I'm not going to be a jerk. I'm not going to be a jerk. I'm not going to be a jerk. (then) And you're not going to be a jerk. And you're not going to be a jerk. I'm not going to be a jerk. And I'm not going to be a jerk. You are not going to be a jerk. You are not going to be a jerk. <b> </b> <b> </b> <b> </b> <b> </b> <b> 7. </b> The door is slammed hard, and a tall, dark figure with a fedora is flung through it. He is, indeed, the stranger. <b> INT. LOBBY, HOTEL LOUNGE - EVENING </b> The man in the fedora is being buffeted by the hotel guests, and he see

IndexError: ignored

### Note: The above code generates repetitive output (tested ~4 times)

Let's try **top_k = 5**:

In [85]:
model.eval()

NUMBER_OF_HALF_MOVIE_SCRIPT_LINES_TO_GENERATE = 10

genre_name = "COMEDY"
prompt = genre_name + " | "

for i in range(0, NUMBER_OF_HALF_MOVIE_SCRIPT_LINES_TO_GENERATE):
  # debug prints below
  #print("prompt:")
  #print(prompt)
  generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
  generated = generated.to(device)

  # debug prints below
  #print(generated)

  sample_outputs = model.generate(
                                generated, 
                                #bos_token_id=random.randint(1,30000),
                                do_sample=True,   
                                top_k=5, 
                                max_length=768,
                                top_p=0.95, 
                                num_return_sequences=1
                                )

  sample_output_decoded = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
  # debug prints below
  #print("sample_output_decoded:")
  #print(sample_output_decoded)
  sample_output_decoded_parts = sample_output_decoded.split(genre_name)
  # debug prints below
  #print("sample_output_decoded_parts:")
  #print(sample_output_decoded_parts)
  #print("len(sample_output_decoded_parts):")
  #print(len(sample_output_decoded_parts))
  prompt = sample_output_decoded_parts[1][2:] + " " + genre_name + " | " # the sample_output_decoded_parts[1][2:] gets rid
                                                                         # of the " | "

  print(sample_output_decoded_parts[0] + genre_name + sample_output_decoded_parts[1])
  
  if (len(sample_output_decoded_parts) == 2): # the midway genre tag wasn't generated
    sample_output_decoded_tokens = sample_output_decoded.split(" ")
    sample_output_decoded_tokens_length = len(sample_output_decoded_tokens)
    half_sample_output_decoded_tokens_length = sample_output_decoded_tokens_length // 2
    prompt_before_genre_name_and_vertical_line = ""
    for token in sample_output_decoded_tokens[:half_sample_output_decoded_tokens_length]:
      prompt_before_genre_name_and_vertical_line += token + " "
    # debug prints below
    #print("prompt_before_genre_name_and_vertical_line:")
    #print(prompt_before_genre_name_and_vertical_line)
    if (prompt_before_genre_name_and_vertical_line != genre_name + " | "):
      prompt = prompt_before_genre_name_and_vertical_line + genre_name + " | " # avoids duplicating the genre tag and the vertical line
    else:
      prompt = genre_name + " | "

# print the last part of the last generated line
print(genre_name + " | " + sample_output_decoded_parts[2])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


COMEDY |  


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


COMEDY | iz, you know, I mean, you know, you know, you know... <b> </b><b> </b> [The two of them are standing in front of the building. They have obviously just come out.] <b> </b><b>? 651? </b><b> </b><b>? 652? </b><b> </b><b> JANE </b><b> </b> (tentatively) <b> </b> Hello. <b> </b><b> MARK </b><b> </b> Hello, Jane. <b> </b><b> JANE </b><b> </b> Hello, Mark. <b> </b><b> MARK </b><b> </b> (tentatively) <b> </b> Hello, Jane. <b> </b><b> JANE </b><b> </b> Good morning, Mark. <b> </b> (to her) <b> </b> Hello, Mark. <b> </b><b> MARK </b><b> </b> Hello, Jane. <b> </b><b> JANE </b><b> </b> Good morning. <b> </b><b> MARK </b><b> </b> Good morning, Jane. <b> </b><b> </b> They both start walking toward the entrance, but Mark is not in front of them. He is standing in the entrance way between two doors. <b> </b><b> JANE </b><b> </b> Mark, I'm sorry to bother you. <b> </b><b> MARK </b><b> </b> Yes. <b> </b> (he stops in front of her, and looks at her) <b> </b> You know, I think we should get goin

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

 iz, you know, I mean, you know, you know, you know... <b> </b><b> </b> [The two of them are standing in front of the building. They have obviously just come out.] <b> </b><b>? 651? </b><b> </b><b>? 652? </b><b> </b><b> JANE </b><b> </b> (tentatively) <b> </b> Hello. <b> </b><b> MARK </b><b> </b> Hello, Jane. <b> </b><b> JANE </b><b> </b> Hello, Mark. <b> </b><b> MARK </b><b> </b> (tentatively) <b> </b> Hello, Jane. <b> </b><b> JANE </b><b> </b> Good morning, Mark. <b> </b> (to her) <b> </b> Hello, COMEDY | ian. <b> </b> Mark turns round. <b> </b><b> MARK </b><b> </b> Hello. <b> </b><b>? 652? </b><b> </b><b> JANE </b><b> </b> Hello. <b> </b> (to Mark) <b> </b> Hello, Mark. <b> </b><b> MARK </b><b> </b> Good morning. <b> </b><b> JANE </b><b> </b> Hello. <b> </b> (to the two of them) <b> </b> Hello, Mark. <b> </b> They all look at her. <b> </b><b> MARK </b><b> </b> Good morning. <b> </b><b> JANE </b><b> </b> Good morning. <b> </b><b> MARK </b><b> </b> Good morning, Amy. <b> </b> They all

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 iz, you know, I mean, you know, you know, you know... <b> </b><b> </b> [The two of them are standing in front of the building. They have obviously just come out.] <b> </b><b>? 651? </b><b> </b><b>? 652? </b><b> </b><b> JANE </b><b> </b> (tentatively) <b> </b> Hello. <b> </b><b> MARK </b><b> </b> Hello, Jane. <b> </b><b> JANE </b><b> </b> Hello, Mark. <b> </b><b> MARK </b><b> </b> (tentatively) <b> </b> Hello, Jane. <b> </b><b> JANE </b><b> </b> Good morning, Mark. <b> </b> (to her) <b> </b> Hello, COMEDY | ian. <b> </b> Mark turns round. <b> </b><b> MARK </b><b> </b> Hello. <b> </b><b>? 652? </b><b> </b><b> JANE </b><b> </b> Hello. <b> </b> (to Mark) <b> </b> Hello, Mark. <b> </b><b> MARK </b><b> </b> Good morning. <b> </b><b> JANE </b><b> </b> Hello. <b> </b> (to the two of them) <b> </b> Hello, Mark. <b> </b> They all look at her. <b> </b><b> MARK </b><b> </b> Good morning. 
 ian. <b> </b> Mark turns round. <b> </b><b> MARK </b><b> </b> Hello. <b> </b><b>? 652? </b><b> </b><b> JANE 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 ian. <b> </b> Mark turns round. <b> </b><b> MARK </b><b> </b> Hello. <b> </b><b>? 652? </b><b> </b><b> JANE </b><b> </b> Hello. <b> </b> (to Mark) <b> </b> Hello, Mark. <b> </b><b> MARK </b><b> </b> Good COMEDY | iz! <b> </b><b> JANE </b><b> </b> You are? <b> </b><b> MARK </b><b> </b> Yes. <b> </b><b> JANE </b><b> </b> Oh! <b> </b><b> MARK </b><b> </b> Well, you know, I've got some new stuff to do. <b> </b><b> JANE </b><b> </b> Oh, I see. Well - well - well - well - well - well! <b> </b><b> MARK </b><b> </b> Well - well, well, well - I hope you like your new stuff. <b> </b><b> JANE </b><b> </b> Oh! <b> </b> (turning to Mark) <b> </b> Well, Mark! You're really great. <b> </b><b> MARK </b><b> </b> Well, I've got to go now. <b> </b><b> JANE </b><b> </b> Well, I'm certainly going. I have to get back to the office. <b> </b><b> MARK </b><b> </b> No, I don't think so. <b> </b><b> JANE </b><b> </b> Well, then I'll see you later then. <b> </b><b> MARK </b><b> </b> Oh! <b> </b> He walks out of 

### Note: The above code generates repetitive output (tested ~4 times)

Let's try with **top_k = 1**:

In [88]:
model.eval()

NUMBER_OF_HALF_MOVIE_SCRIPT_LINES_TO_GENERATE = 10

genre_name = "COMEDY"
prompt = genre_name + " | "

for i in range(0, NUMBER_OF_HALF_MOVIE_SCRIPT_LINES_TO_GENERATE):
  # debug prints below
  #print("prompt:")
  #print(prompt)
  generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)
  generated = generated.to(device)

  # debug prints below
  #print(generated)

  sample_outputs = model.generate(
                                generated, 
                                #bos_token_id=random.randint(1,30000),
                                do_sample=True,   
                                top_k=1, 
                                max_length=768,
                                top_p=0.95, 
                                num_return_sequences=1
                                )

  sample_output_decoded = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
  # debug prints below
  #print("sample_output_decoded:")
  #print(sample_output_decoded)
  sample_output_decoded_parts = sample_output_decoded.split(genre_name)
  # debug prints below
  #print("sample_output_decoded_parts:")
  #print(sample_output_decoded_parts)
  #print("len(sample_output_decoded_parts):")
  #print(len(sample_output_decoded_parts))
  prompt = sample_output_decoded_parts[1][2:] + " " + genre_name + " | " # the sample_output_decoded_parts[1][2:] gets rid
                                                                         # of the " | "

  print(sample_output_decoded_parts[0] + genre_name + sample_output_decoded_parts[1])
  
  if (len(sample_output_decoded_parts) == 2): # the midway genre tag wasn't generated
    sample_output_decoded_tokens = sample_output_decoded.split(" ")
    sample_output_decoded_tokens_length = len(sample_output_decoded_tokens)
    half_sample_output_decoded_tokens_length = sample_output_decoded_tokens_length // 2
    prompt_before_genre_name_and_vertical_line = ""
    for token in sample_output_decoded_tokens[:half_sample_output_decoded_tokens_length]:
      prompt_before_genre_name_and_vertical_line += token + " "
    # debug prints below
    #print("prompt_before_genre_name_and_vertical_line:")
    #print(prompt_before_genre_name_and_vertical_line)
    if (prompt_before_genre_name_and_vertical_line != genre_name + " | "):
      prompt = prompt_before_genre_name_and_vertical_line + genre_name + " | " # avoids duplicating the genre tag and the vertical line
    else:
      prompt = genre_name + " | "

# print the last part of the last generated line
print(genre_name + " | " + sample_output_decoded_parts[2])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

COMEDY |  
COMEDY |  
COMEDY |  
COMEDY |  
COMEDY |  
COMEDY |  


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


COMEDY |  
COMEDY |  
COMEDY |  
COMEDY |  


IndexError: ignored

### Note: The above code generates garbage output (tested ~3 times)